# Imports

In [ ]:
import numpy as np
import sys
import os
#%load_ext autoreload

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.models.yamnet_train import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

2025-12-09 22:48:50.444578: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-09 22:48:50.444942: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-09 22:48:50.487926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-09 22:49:17.236738: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

Using: cpu


In [3]:
from loguru import logger
import sed_eval

gt_pkl_path = 'data/processed/yamnet/spectrograms_train_list.pkl'

gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':1}]
                    for ref_event in gt_events}

detection_classes = [1]

def calculate_metrics(pred_event_dict, gt_event_dict, time_resolution=1.0, t_collar=0.25):

    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(detection_classes, time_resolution=time_resolution)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        segment_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    #print(segment_based_metrics)

    return segment_based_metrics.results_overall_metrics()['f_measure']

In [ ]:
# manual train test split (stratified)
np.random.seed(0)
data = pickle.load(open('data/processed/yamnet/spectrograms_train.pkl', 'rb'))
train_size = 0.8
train_idx = []
for label in np.unique(data['event_label']):
    choices = np.where(data['event_label'] == label)[0]
    train_idx.append(np.sort(np.random.choice(choices, size = int(np.round(len(choices)*train_size)), replace = False)))
train_idx = np.sort(np.concatenate(train_idx))

# uncomment to retrain
"""
solver = Solver(epochs = 5, train_idx=train_idx, path_prefix = path_prefix, device = device, batch_size = 8)
solver.train()"""

'\nsolver = Solver(epochs = 5, train_idx=train_idx, path_prefix = path_prefix, device = device, batch_size = 8)\nsolver.train()'

In [5]:
all_indices = list(range(len(data["event_label"])))
all_files = [f"{DETECTION_TRAIN_PATH}/train_snipped_scene_{str(i).zfill(4)}.wav" for i in all_indices]
val_idx = [i for i in range(len(data["event_label"])) if i not in train_idx]
val_files = [all_files[i] for i in val_idx]

# Test

In [8]:
test_path = Path("data") / "processed" / "yamnet" / "spectrograms_test.pkl"
test_data = pickle.load(open(test_path, "rb"))
filepaths = [os.path.join(DETECTION_TEST_PATH, file) for file in test_data['files']]
events = run_yamnet(filepaths)

2025-12-09 22:50:10.175 | INFO     | __main__:run_yamnet:10 - Loading from checkpoint checkpoints/yamnet_detector.pth...
2025-12-09 22:50:17.440 | INFO     | src.models.yamnet_train:load_model:188 - Loaded model weights from checkpoints/yamnet_detector.pth


Pre-computing features for 500 files...


Testing...: 100%|██████████| 500/500 [00:06<00:00, 79.73it/s]


In [9]:
events['test_snipped_scene_0000.wav']

[{'file': 'test_snipped_scene_0000.wav',
  'event_onset': 1.92,
  'event_offset': 5.76,
  'ground_truth': 'siren',
  'match': None}]

In [10]:
gt_pkl_path = 'data/processed/yamnet/spectrograms_test_list.pkl'

gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':1}]
                    for ref_event in gt_events}

pred_event_dict = {}
for key, value in events.items():
    for value_i in range(len(value)):
        value[value_i] = value[value_i] | {
            "event_label": 1
        }
    pred_event_dict[key] = value

In [11]:
from loguru import logger
import sed_eval

detection_classes = [1]

def calculate_metrics(pred_event_dict, gt_event_dict, time_resolution=1.0):

    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(detection_classes, time_resolution=time_resolution)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        segment_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    #print(segment_based_metrics)

    return segment_based_metrics.results_overall_metrics()['f_measure']

segment_based_metrics = calculate_metrics(pred_event_dict, gt_event_dict)
segment_based_metrics

{'f_measure': 0.8960910440376051,
 'precision': 0.9253960143076136,
 'recall': 0.8685851318944844}

# Cut audio

In [ ]:
from main import cut_events_from_audio

In [ ]:
cut_events_from_audio((Path("data") / "processed" / "yamnet" / "extracted_audio"), events)

2025-12-06 11:28:27.026 | INFO     | __main__:cut_events_from_audio:5 - Created directory: data/processed/yamnet/extracted_audio
100%|██████████| 500/500 [00:04<00:00, 100.53it/s]
